In [2]:
import collections 
from itertools import chain
import torch 
import time 

from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter

## 项目folder
from dataset import SeqMultiLabelDataset
from process import Schema2Label
from evaluation import multilabel_inference
from models import BertSlotClassifier

## 通用folder
from src.dataset.converter import data_loader 
from src.train_utils import set_seed, ModelSave, get_torch_device, EarlyStop, TrainParams
from src.models.bert import BertClassifier
from src.metric import multilabel_metrics, multilabel_log
import transformers 
from transformers import BertTokenizer
transformers.logging.set_verbosity_error()
device = get_torch_device()
set_seed(1234)

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [3]:
DIR = './trainsample'
schema = Schema2Label(DIR+'/duee_event_schema.json')

tp = TrainParams(
    log_steps = 50,
    save_steps = 10,
    epoch_size=20,
    max_seq_len=512,
    batch_size=16,
    loss_fn=nn.BCEWithLogitsLoss(),
    lr=1e-5,
    weight_decay=0.0,
    epsilon=1e-6,
    warmup_steps=100,
    dropout_rate=0.2,
    gradient_clip=5.0,
    early_stop_params = {
        'monitor':'acc',
        'mode':'max',
        'min_delta': 0,
        'patience':3,
        'verbose':False
    },
    slot_positions = [1,8,12,16,20,24,28,32,36,40,44,48,52,56,60,64,68,72,76,80,84,88,92,96,100,104,110,114,118,122,126,130,134,138,142,146,150,154,158,162,166,170,174,178,182,186,190,194,198,202,206,212,216,220,224,230,234,238,242,246,250,254,258,262,266],
    pretrain_model = 'bert-base-chinese',
    continue_train=False,
    label2idx = schema.event_label,
    idx2label = {j:i for i,j in schema.event_label.items()},
    label_size=len(schema.event_label)
)


In [5]:
tokenizer = BertTokenizer.from_pretrained(tp.pretrain_model, do_lower_case=True)
special_tokens_dict = {'additional_special_tokens': ['[E1]','[E2]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

train_dataset = SeqMultiLabelDataset(data_loader(DIR+'/train_event_slot.txt'),
                                tokenizer, tp.max_seq_len, tp.label2idx)
valid_dataset = SeqMultiLabelDataset(data_loader(DIR+'/valid_event_slot.txt'), 
                                tokenizer, tp.max_seq_len,  tp.label2idx)
train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)
train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=tp.batch_size)
valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=tp.batch_size)

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [6]:
tp.update({'num_train_steps': int(len(train_loader)*tp.epoch_size)})

CKPT = './checkpoint/event'
saver = ModelSave(CKPT, continue_train=False)
saver.init()
es = EarlyStop(**tp.early_stop_params)
global_step = 0
tb = SummaryWriter(CKPT)

model = BertSlotClassifier(tp)
model.bert.resize_token_embeddings(len(tokenizer))
model.to(device)
optimizer, scheduler = model.get_optimizer()

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

In [7]:
for epoch_i in range(tp['epoch_size']):
    print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10}  | {'Elapsed':^9}")
    print("-"*60)

    # Measure the elapsed time of each epoch
    t0_epoch, t0_batch = time.time(), time.time()
    total_loss, batch_loss, batch_counts = 0, 0, 0

    model.train()
    for step, batch in enumerate(train_loader):
        global_step +=1
        batch_counts +=1

        #Forward propogate
        model.zero_grad()
        feature = {k:v.to(device) for k, v in batch.items()}
        logits = model(feature)
        loss = model.compute_loss(feature, logits)
        batch_loss += loss.item()
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), tp.gradient_clip)
        optimizer.step()
        scheduler.step()
        # Log steps for train loss logging
        if (step % tp.log_steps == 0 and step != 0) or (step == len(train_loader) - 1):
            time_elapsed = time.time() - t0_batch
            print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^9} | {time_elapsed:^9.2f}")
            tb.add_scalar('loss/batch_train', batch_loss / batch_counts, global_step=global_step)
            batch_loss, batch_counts = 0, 0
            t0_batch = time.time()

    # On Epoch End: calcualte train & valid loss and log overall metrics
    time_elapsed = time.time() - t0_epoch
    val_metrics = multilabel_metrics(model, valid_loader, device)
    avg_train_loss = total_loss / step

    print("-"*70)
    print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_metrics['val_loss']:^10.6f} | {time_elapsed:^9.2f}")
    multilabel_log(epoch_i, val_metrics)
    print("\n")
    if es.check(val_metrics):
        break 

 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   50    |   0.231221   |     -     |   43.12  
   1    |   100   |   0.083548   |     -     |   41.28  
   1    |   150   |   0.078299   |     -     |   41.30  
   1    |   200   |   0.041807   |     -     |   41.32  
   1    |   250   |   0.026018   |     -     |   41.31  
   1    |   300   |   0.018384   |     -     |   41.32  
   1    |   350   |   0.019209   |     -     |   41.30  
   1    |   400   |   0.016319   |     -     |   41.29  
   1    |   450   |   0.014062   |     -     |   41.27  
   1    |   500   |   0.012857   |     -     |   41.33  
   1    |   550   |   0.011574   |     -     |   41.32  
   1    |   600   |   0.011120   |     -     |   41.28  
   1    |   650   |   0.010457   |     -     |   41.29  
   1    |   700   |   0.010218   |     -     |   41.30  
   1    |   744   |   0.010285   |     -     |   35.79  
-------------------------

In [8]:
from evaluation import multilabel_evaluation, extract_multilabel, multilabel_inference
test_dataset = SeqMultiLabelDataset(data_loader(DIR+'/test_event_slot.txt'), 
                                tokenizer, tp.max_seq_len,  tp.label2idx)
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset, sampler=test_sampler, batch_size=tp.batch_size)

In [9]:
test = pd.DataFrame(test_dataset.raw_data)
valid = pd.DataFrame(valid_dataset.raw_data)
test['prob'] = multilabel_inference(model, test_loader, device)
valid['prob'] = multilabel_inference(model, valid_loader, device)
valid['pred_label'] = valid['prob'].map(lambda x: extract_multilabel(x, tp.idx2label, 0.5))
test['pred_label'] = test['prob'].map(lambda x: extract_multilabel(x, tp.idx2label, 0.5))
valid.to_csv(DIR +'/valid_event_slot_pred.csv')
test.to_csv(DIR +'/test_event_slot_pred.csv')

In [10]:
multilabel_evaluation(valid['label'].values, valid['pred_label'].values, True)

   n_sample  n_pos  precision   recall       f1  accuracy
0      1492   1651   0.948918  0.95639  0.95264  0.909562
